<a href="https://colab.research.google.com/github/MirandaCarou/Quantum-Machine-Learning/blob/main/QML_WINE_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[ PRÁCTICA QML ]** -   **Miranda Carou Laiño**

In [1]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit_algorithms
!pip install qiskit_machine_learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=c455e171db5e61567e1df71bb849619816673d8f45a1e2a070d61c377d41e303
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

*The dataset is composed of 178 instances, each of them composed of 13 features
(alcohol, malic acid, ash, etc.) and a target label representing one of each of
the three possible classes of wine.*

In [2]:
import time
from qiskit import *
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap, ZFeatureMap, StatePreparation, Initialize
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import COBYLA, ADAM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn . datasets import load_wine
from qiskit_machine_learning.algorithms import VQC
from sklearn.metrics import accuracy_score
from qiskit_machine_learning.utils import algorithm_globals

import math


wine_data = load_wine ()
features = wine_data . data
labels = wine_data . target

# **Preprocesamiento**


La etapa de preprocesamiento de los datos es una de las etapas más importantes y pueden marcar la diferencia entre un buen modelo y un mal modelos. Dentro de area de Machine Learning, la etapa de preprocesamiento de datos se puede dividir en 3 partes princiapales:


1.   **Integración de los datos**. Unificación de datos datos combinados de diferentes fuentes para ofrecer al usuario un punto de vista de los datos
2.   **Limpieza de los datos**. Proceso de detección y correción o eliminación de datos incorrectos o redundantos.
3.   **Transformaicón de los datos**: Procesor de convertir los datos a un formato más adecuado para el tratamiento y análisis.






In [3]:
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_normalized, labels, test_size=0.2, random_state=42) #80% entrenamiento 20% prueba

print("Training features shape:", X_train.shape)
print("Test features shape:", X_test.shape)
print("Training labels shape:", y_train.shape)
print("Test labels shape:", y_test.shape)



Training features shape: (142, 13)
Test features shape: (36, 13)
Training labels shape: (142,)
Test labels shape: (36,)


# **Data Encoding**

La codificación de los datos tambien es algo crucial en QML y suele recibir los nombres de "Data enncoding", "Data Embeding" o "Loading". Es un proceso que consiste principalmente en mapear los datos clásicos a datos cuanticos, unos pasos posteriores que ya dependen más del algoritmo que vayas a implementar y finalmente realizar las medidas y mapear los datos quanticos a clasicos.

Existen diferentes métodos de data embeding:

1. **Basis encoding**: Es el método más simple. Asocias un cadena de N-bit clásicos con unas bases computaiconales. Util cuando trabajas con numeros en binario.
2. **Amplitude Encoding** Método en el que codificas los datos clásicos en las amplitudes de los estaods pero necesita que anteriormente tengas ese vector normalizado
3. **Angle encoding**: Método donde condificas los datos clásicos como rotaciones de los angulos de los qubits pero en este caso solo codifcas uno en uno , no todo el dataset entero de golpe. Este método es bueno pra evitar problemas de decoherencia.
4. **Feature maps**: (Lo especificados para esta práctica). Donde tenemos 3 opciones posibles:

  *   **Codificación Arbitraría** Codifica de manera arbitraria N caracterissticas como rotaciones de N puertas parametrizadas en n qubits y solo codificas de cada vez un punto. Usa una constante n que indica la profundidad del circuito
  *   **Feature maps** Mapea un vector de caracteristicas clásico en un vector cuántico del espcio de Hlbert.

  Ejemplos:

  1.   Circuitos de codificación: **PauliFeatureMap, ZFeatureMap, ZZFeatureMap**
  2.   Circuitos de preparacion de datos : **StatePreparation, Initialize**

  En el contexto de esta práctica, solo usaré como prueba, los circuitos de codificación porque al final, a diferencia de los otros, estos tienen como proposito codificar los datos en estados cuánticos para usar en algoritmos de machine learning y ya se pasan como parámetros de circuitos parametrizados.

    *   **PauliFeatureMap** Usa combinaciones de operadores de Pauli. Es más recomendable para capturar interaciones complejas en los datos (*Deprecated since version 1.3_pending*)
    *   **ZFeatureMap**: Codifica los datos usando únicamente operadores de Pauli Z. E muy simple y rápido, muy bueno para datos con poca complejidad
    *   **ZZFeatureMap**: Codifica los datos usando interacciones de segundo orden entre qubits. Muy bueno cuando las correlaciones entre las caracteristicas del dataset con algo importante.

  En el contexto de nuestro dataset parece que la mejor opción es ZZFeatureMap ya que vamos a clasificar vinos en función de sus propiedades químicas





**ZZFeautreMap**

In [20]:
zz_feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=2)

**ZFeatureeMap**

In [22]:
z_feature_map = ZFeatureMap(X_train.shape[1], reps=2, insert_barriers=True)

# **Encontrar el Ansatz apropiado**


Para seguir contruyendo nuestro modelos necesitamos encontrar el estado de referencia por el que vamos a empezar a enfrentar el problema. Podemos elegirlo manualmente o usar un circuito plantilla como lo son por ejemplo: **TwoLocal** , **RealAmplitude** o **EfficientSU2**

- *Escogo siempre el tipo de entralazamiento lineal asi de primeras porque sobrecarga menos el circuito, es más rápido y menos propensos a errores*

**RealAmplitudes**

En mi sincera opinión, creo que este es el más popular para problemas de clasificación y quantum machine learning. Es simple y eficaz. Se basa en puertas rotacionales RY y entrelazamientos entre qubits.

*Creo que para nuestro dataset es el que mejor podría manejar las correlaciones y no ser muy demandante para mi ordenador.*

In [33]:
ansatz_amplitudes = RealAmplitudes(num_qubits=X_train.shape[1],
                        entanglement='linear',
                        reps=2,
                        insert_barriers=True)

**TwoLocal**

Ese ya es un circuito más genéricos y personalizable que combina las puertas rotacionales y las de entrelazamiento. Es una opción mucho más versátil que puede ser muy interesante pero tambien es una opción mucho mas dificil de optimizar y va a consumirte muchos mas recursos del ordenador, que si no tienes un ordenador muy potente esto puede jugar en tu contra.

In [32]:
ansatz_twolocal = TwoLocal(num_qubits=X_train.shape[1],
                           rotation_blocks='ry',
                           entanglement_blocks='cz',
                           reps=2,
                           insert_barriers=True)

**EfficientSU2**

Esta es la optión más ligera para utilizar pero también la más limitada pero si tienes muchas limitaciones de hardware es la mejor opción aunque va a tener más dificultades para encontrarte correlaciones complejas en el dataset. Se basa en puertas RY y Rz para las rotaciones y puertas CNOT para los acoplamientos.


In [31]:
ansatz_su2 = EfficientSU2(num_qubits=X_train.shape[1],
                          reps=2,
                          insert_barriers=True)

# **Optimizador**

Encontrar un buen optimizador tambien es parte esencial de todo algoritmo variacional y que es lo que se utiliza para evaluar y seleccionar los parametros del siguiente recorrido del bucle variacional, al menos hasta encontrar un estado estable.

A grandes escalas hay dos tipos de optimizadores:

1.   **Locales**: Buscan el punto donde se minimice la función de coste, empezando desde el punto incial y luego se va moviendo en base a lo que va observando dentro de la región donde esta y no pueden ver más alla de su region. Son optimizadores que por lo general convergen rápido pero que tambien dependen del punto inicial donde empiecen. Muy sensible a minimos locales

2.   **Globales** : Buscan el punto donde se minimiza la función de coste pero en distintas regiones. Es menos sensible que el anterior a minimos locales pero converge mucho mas lento.Obviamente tambien es una opción que requiere muchos más recursos computacionales (*En principio no me remplanteo esta opción por ser las más lenta y pesada*)

3.  **Gradient-Based**: Busca el mínimo de la función de coste navegando por el campo de perdidas. Calcula el gradiente de la función de pérdidas con respecto a los parámentros. Es muy eficiente para espacios suaves pero clacular gradientes suele ser costoso en cuántica  y además es muy sensible a inicializar mal los parámetros.  

4. **Libre de Gradiente**: No necesita calcular grandiente, en vez de eso, evalua la función de pérdida en diferentes puntos. Esta es una opción bastante funcional y buena si tienes problemas de ruido. Si que es verdad que no es muy eficiente para problemas de muchas variables. (*No voy a considerar esta opción como prueba porque al no estar utilizando hardware real muy sensible al rudio y puedo calcular gradientes, no me tiene sentido*)


Mis 3 posibles opciones para pruebas son:

*   **COBYLA** (optimizador local): Opción simple y eficiente si el circuito no es muy complejo
*   **Adam** (basado en gradiente): Ya conocido y utilizado de la practica anterior







**COBYLA**

In [45]:
optimizer_cobyla = COBYLA(maxiter=500, tol=1e-3)

**ADAM**

In [46]:
optimizer_adam = ADAM(maxiter=500, lr=0.01)

# **Quantum Neural Network Models**

**Modelos más secillos y rápidos**

*Mi intención es intentar validar primero el flujo general del modelo y luego ya después probar configuraciones más avanzadas*

  - **MODELO-1**

    *   ZFeatureMap
    *   RealAmplitudes
    *   COBYLA

In [54]:
z_feature_map_1 = ZFeatureMap(X_train.shape[1], reps=2, insert_barriers=True)
ansatz_amplitudes_1 = RealAmplitudes(num_qubits=X_train.shape[1],
                        entanglement='linear',
                        reps=2)
optimizer_cobyla_1 = COBYLA(maxiter=50, tol=1e-3)


vqc = VQC(feature_map=z_feature_map_1, ansatz=ansatz_amplitudes_1, optimizer=optimizer_cobyla_1)

start = time.time()
vqc.fit(X_train, y_train)
elapsed_time = time.time() - start
print(f"Tiempo de entrenamiento: {elapsed_time} segundos")

train_score = vqc.score(X_train, y_train)
test_score = vqc.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 1454.4215223789215 segundos
Precisión del VQC en el conjunto de entrenamiento: 0.5
Precisión del VQC en el conjunto de prueba: 0.5833333333333334


In [57]:
elapsed_time_minutes = elapsed_time / 60  # Convertir segundos a minutos
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.2f} minutos")

Tiempo de entrenamiento: 24.24 minutos


- **MODELO-2**

  *   **ZFeatureMap**: 2 repeticiones
  *   **EfficientSU2**: 1 repetición
  *   **COBYLA**: 300 interacciones (*Igual 100 sería mejor?*)

In [58]:
z_feature_map_2 = ZFeatureMap(X_train.shape[1], reps=2)
ansatz_su2_2 = EfficientSU2(num_qubits=X_train.shape[1],
                            reps=1)
optimizer_cobyla_2 = COBYLA(maxiter=100, tol=1e-3)


vqc_2 = VQC(feature_map=z_feature_map_2, ansatz=ansatz_su2_2, optimizer=optimizer_cobyla_2)

start = time.time()
vqc_2.fit(X_train, y_train)
elapsed_time = time.time() - start
elapsed_time_minutes = elapsed_time / 60
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.3f} minutos")

train_score = vqc_2.score(X_train, y_train)
test_score = vqc_2.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 49.059 minutos
Precisión del VQC en el conjunto de entrenamiento: 0.5985915492957746
Precisión del VQC en el conjunto de prueba: 0.6944444444444444


**Modelo algo más robusto e igual más baloanceado**

*Siguen siendo modelos simples pero que creo que pueden ya modelar relaciones batante relevantes. Aquí voy a probar con menos iteraciones que empiezo a percibir tiempo de ejecución muy largos*

- **MODELO-3**

  *   **ZZFeatureMap**: 2 repeticiones
  *   **RealAmplitudes**: 2 repetición
  *   **COBYLA**: 100 interacciones

In [59]:
zz_feature_map_3 = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=2)
ansatz_amplitudes_3 = RealAmplitudes(num_qubits=X_train.shape[1],
                        entanglement='linear',
                        reps=2)
optimizer_cobyla_3 = COBYLA(maxiter=100, tol=1e-3)

vqc_3 = VQC(feature_map=zz_feature_map_3, ansatz=ansatz_amplitudes_3, optimizer=optimizer_cobyla_3)


start = time.time()
vqc_3.fit(X_train, y_train)
elapsed_time = time.time() - start
print(f"Tiempo de entrenamiento: {elapsed_time} segundos")

train_score = vqc_3.score(X_train, y_train)
test_score = vqc_3.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 4254.173606157303 segundos
Precisión del VQC en el conjunto de entrenamiento: 0.4788732394366197
Precisión del VQC en el conjunto de prueba: 0.6111111111111112


In [60]:
elapsed_time_minutes = elapsed_time / 60  # Convertir segundos a minutos
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.2f} minutos")

Tiempo de entrenamiento: 70.90 minutos




**Modelo algo más complejo**

*Esta combinación es más poderosa para capturar patrones complejos en los datos, pero podría ser más lenta de optimizar*

  -  **MODELO-4**
     *  **ZZFeatureMap**: 2 repeticiones
     *  **TwoLocal**: 2 repetición
     *  **Adam**: 100 interacciones



In [61]:
zz_feature_map_4 = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=2)
ansatz_twolocal = TwoLocal(num_qubits=X_train.shape[1],
                           rotation_blocks='ry',
                           entanglement_blocks='cz',
                           reps=2)
optimizer_adam = ADAM(maxiter=100, lr=0.01)


vqc_4 = VQC(feature_map=zz_feature_map_3, ansatz=ansatz_amplitudes, optimizer=optimizer_cobyla_3)


start = time.time()
vqc_4.fit(X_train, y_train)
elapsed_time = time.time() - start
elapsed_time_minutes = elapsed_time / 60
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.2f} minutos")

train_score = vqc_4.score(X_train, y_train)
test_score = vqc_4.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 70.75 minutos
Precisión del VQC en el conjunto de entrenamiento: 0.5070422535211268
Precisión del VQC en el conjunto de prueba: 0.6111111111111112


Ahora despues de ver estos resultamos podemos probar combinaciones diferentes haber si conseguimos mejorar los resultados

*Vamos a probar el modelo 2 pero con otro ansatz a ver que ocurre ya que es el modelo que de momento ha mostrado el mejor resultado*

  *  **MODELO-5**
      *  **ZZFeatureMap**: 2 repeticiones
      *  **TwoLocal**: 3 repetición
      *  **COBYLA**: 100 interacciones


In [62]:
z_feature_map_5 = ZFeatureMap(X_train.shape[1], reps=2)
ansatz_twolocal_5 = TwoLocal(num_qubits=X_train.shape[1],
                           rotation_blocks='ry',
                           entanglement_blocks='cz',
                           reps=3)
optimizer_cobyla_5 = COBYLA(maxiter=100, tol=1e-3)


vqc_5 = VQC(feature_map=z_feature_map_5, ansatz=ansatz_twolocal_5, optimizer=optimizer_cobyla_5)

start = time.time()
vqc_5.fit(X_train, y_train)
elapsed_time = time.time() - start
elapsed_time_minutes = elapsed_time / 60
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.3f} minutos")

train_score = vqc_5.score(X_train, y_train)
test_score = vqc_5.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 61.238 minutos
Precisión del VQC en el conjunto de entrenamiento: 0.5845070422535211
Precisión del VQC en el conjunto de prueba: 0.6666666666666666




*Vamos a probar con un ansatz  distinto*

  *  **MODELO-6**
      *  **ZZFeatureMap**: 2 repeticiones
      *  **RealEstimation**: 3 repetición
      *  **COBYLA**: 100 interacciones



In [63]:
z_feature_map_6 = ZFeatureMap(X_train.shape[1], reps=2)
ansatz_amplitudes_6 = RealAmplitudes(num_qubits=X_train.shape[1],
                        entanglement='linear',
                        reps=3)
optimizer_cobyla_6 = COBYLA(maxiter=100, tol=1e-3)


vqc_6 = VQC(feature_map=z_feature_map_6, ansatz=ansatz_amplitudes_6, optimizer=optimizer_cobyla_6)

start = time.time()
vqc_6.fit(X_train, y_train)
elapsed_time = time.time() - start
elapsed_time_minutes = elapsed_time / 60
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.3f} minutos")

train_score = vqc_6.score(X_train, y_train)
test_score = vqc_6.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 55.003 minutos
Precisión del VQC en el conjunto de entrenamiento: 0.6408450704225352
Precisión del VQC en el conjunto de prueba: 0.7777777777777778




*Tiene buena pinta, vamos a subir las repeticiones del feature map a ver si eso ayuda algo y tambien las interacciones máximas del cobyla*

  *  **MODELO-8**
     *   **ZFeatureMap**: 3 repeticiones
     *   **RealEstimation**: 3 repetición
     *   **COBYLA**: 150 interacciones



In [5]:
z_feature_map_7 = ZFeatureMap(X_train.shape[1], reps=3)
ansatz_amplitudes_7 = RealAmplitudes(num_qubits=X_train.shape[1],
                        entanglement='linear',
                        reps=3)
optimizer_cobyla_7 = COBYLA(maxiter=150, tol=1e-3)


vqc_7 = VQC(feature_map=z_feature_map_7, ansatz=ansatz_amplitudes_7, optimizer=optimizer_cobyla_7)

start = time.time()
vqc_7.fit(X_train, y_train)
elapsed_time = time.time() - start
elapsed_time_minutes = elapsed_time / 60
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.3f} minutos")

train_score = vqc_7.score(X_train, y_train)
test_score = vqc_7.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 76.194 minutos
Precisión del VQC en el conjunto de entrenamiento: 0.6056338028169014
Precisión del VQC en el conjunto de prueba: 0.7222222222222222


*Voy a intentar superar ese 77% con un modelo algo más exigente en cuanto a recursos computacionales y algo más lento *

  *  **MODELO-8**
     *   **ZFeatureMap**: 3 repeticiones
     *   **RealEstimation**: 3 repetición **Full Entanglement**
     *   **COBYLA**: 200 interacciones


In [6]:
z_feature_map_8 = ZFeatureMap(X_train.shape[1], reps=3)
ansatz_amplitudes_8 = RealAmplitudes(num_qubits=X_train.shape[1],
                        entanglement='full',
                        reps=3)
optimizer_cobyla_8 = COBYLA(maxiter=200, tol=1e-3)


vqc_8 = VQC(feature_map=z_feature_map_8, ansatz=ansatz_amplitudes_8, optimizer=optimizer_cobyla_8)

start = time.time()
vqc_8.fit(X_train, y_train)
elapsed_time = time.time() - start
elapsed_time_minutes = elapsed_time / 60
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.3f} minutos")

train_score = vqc_8.score(X_train, y_train)
test_score = vqc_8.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 118.787 minutos
Precisión del VQC en el conjunto de entrenamiento: 0.5915492957746479
Precisión del VQC en el conjunto de prueba: 0.6944444444444444


*Voy a intentar con el modelo 6 y solo más interaccione a ver si así notamos mejoría*

  *  **MODELO-9**
     *   **ZFeatureMap**: 2 repeticiones
     *   **RealEstimation**: 4 repetición Entanglement
     *   **COBYLA**: 100 interacciones


In [5]:
z_feature_map_9 = ZFeatureMap(X_train.shape[1], reps=2)
ansatz_amplitudes_9 = RealAmplitudes(num_qubits=X_train.shape[1],
                        entanglement='linear',
                        reps=4)
optimizer_cobyla_9= COBYLA(maxiter=100, tol=1e-3)


vqc_9 = VQC(feature_map=z_feature_map_9, ansatz=ansatz_amplitudes_9, optimizer=optimizer_cobyla_9)

start = time.time()
vqc_9.fit(X_train, y_train)
elapsed_time = time.time() - start
elapsed_time_minutes = elapsed_time / 60
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.3f} minutos")

train_score = vqc_9.score(X_train, y_train)
test_score = vqc_9.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 54.967 minutos
Precisión del VQC en el conjunto de entrenamiento: 0.6338028169014085
Precisión del VQC en el conjunto de prueba: 0.7777777777777778


*¿Igual el mismo y más iteraciones?*

  *  **MODELO-10**
     *   **ZFeatureMap**: 2 repeticiones
     *   **RealEstimation**: 4 repetición Entanglement
     *   **COBYLA**: 500 interacciones

In [4]:
z_feature_map_10 = ZFeatureMap(X_train.shape[1], reps=2)
ansatz_amplitudes_10 = RealAmplitudes(num_qubits=X_train.shape[1],
                        entanglement='linear',
                        reps=4)
optimizer_cobyla_10 = COBYLA(maxiter=500, tol=1e-3)


vqc_10 = VQC(feature_map=z_feature_map_10, ansatz=ansatz_amplitudes_10, optimizer=optimizer_cobyla_10)

start = time.time()
vqc_10.fit(X_train, y_train)
elapsed_time = time.time() - start
elapsed_time_minutes = elapsed_time / 60
print(f"Tiempo de entrenamiento: {elapsed_time_minutes:.3f} minutos")

train_score = vqc_10.score(X_train, y_train)
test_score = vqc_10.score(X_test, y_test)
print(f"Precisión del VQC en el conjunto de entrenamiento: {train_score}")
print(f"Precisión del VQC en el conjunto de prueba: {test_score}")

Tiempo de entrenamiento: 261.183 minutos
Precisión del VQC en el conjunto de entrenamiento: 0.6619718309859155
Precisión del VQC en el conjunto de prueba: 0.7777777777777778


*¿Será este el límite que se puede alcanzar de precisión?

# **Conclusiones**

Tras todas estas pruebas hemos podido observar que los mejores resultados los han proporcionado los modelos 6 y 9, ambos alcanzando un 77.78% de precisión máxima, utilizando ZFeatureMap, RealAmplitude (con entrelazamiento lineal) y el optimizador COBYLA. El Modelo 10 es un claro ejemplo de que puede que 77,78% sea nuestro límite actual o al menos que ese nivel de precisión no se puede superar solo con un mayor número de iteracciones. Se ha percibido tambien que usar "Full Entanglement" no supone una mayor precisión en este dispositivo de pruebas. También se observa que ZZFeatureMap, no es el Feature Map que más potencial nos puede ofrecer en datasets como el nuestro, al igual que el optimizador Adam.
¿Habré cometido algun error? ¿Habre ajustado de mala manera algún parámetro? Puede ser, soy consciente de que igual no he probado todas las opciones posibles y que podría escaparseme agún detalle importante. Puede que la parametrización o el preprocesamiento de los datos no sea el que más potencial puede sacar de este dataset. Igual tambien he pasado por alto algún optimizador con potencial, igualmente creo que se ha alcanzado un buen reesultado, que aunque no iguala a la precisión de los métodos clásicos es un buen resultado.